In [1]:
import numpy as np
import torch
import dgl
import time

/home/bear/miniconda3/envs/dgl_v09/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 首先每个训练点有一个自己的标签(idx)
# 随后对标签进行传递 dst <-> src
# 在经过n轮之后,每个点都有了一个自己的最终标签
# 之后对聚类结果进行合并

graph = torch.as_tensor(np.fromfile("/raid/bear/data/raw/papers100M/graph1.bin",dtype=np.int32))
trainids = torch.as_tensor(np.fromfile("/raid/bear/data/raw/papers100M/trainIds.bin",dtype=np.int64))
src = graph[::2]
dst = graph[1::2]
src = src.cuda()
dst = dst.cuda()

nodeNUM = 111059956
nodeLabel = torch.zeros(nodeNUM).to(torch.int32) -1
nodeLabel[trainids] = trainids.to(torch.int32)
nodeLabel = nodeLabel.cuda()

In [3]:
hopNUM = 2
for _ in range(hopNUM):
    dgl.lpGraph(src,dst,nodeLabel)

In [4]:
# 清空存储
src = src.cpu()
dst = dst.cpu()
nodeLabel = nodeLabel.cpu()
import gc
torch.cuda.empty_cache()
gc.collect()

46

In [9]:
# 提取训练集标签
trainLabel = nodeLabel[trainids.to(torch.int64)]

# 按照标签进行聚类(数量不定)
labelCluster = torch.bincount(trainLabel)
sortLabelsNUM,labelIdx = torch.sort(labelCluster,descending=True)

In [13]:
bound = len(torch.nonzero(sortLabelsNUM > 0).reshape(-1))

sortLabelsNUM = sortLabelsNUM[:bound]
labelIdx = labelIdx[:bound]

In [5]:
def findMinPart(partInfo):
    min_value, min_index = torch.min(partInfo, dim=0)
    return min_value, min_index

def changeInfo(partInfo,changeIdx,changeValue,acc=True):
    if acc:
        partInfo[changeIdx] += changeValue
    else:
        partInfo[changeIdx] = changeValue

In [15]:
# 将小聚类进行合并，生成为具体的聚类数目
partNUM = 8
partInfo = torch.zeros(partNUM,dtype=torch.int64)
label2part = torch.zeros(torch.max(labelIdx).item()+1,dtype=torch.int64)
for index,labelId in enumerate(labelIdx):
    value,partid = findMinPart(partInfo)    # 查询当前最小
    label2part[labelId] = partid            # 放入
    changeInfo(partInfo,partid,sortLabelsNUM[index])    # 修改当前存储表

# 此时所有的小聚类均放入大聚类中，id: 0 - (partNUM-1)
trainids
trainIdsInPart = label2part[trainLabel.to(torch.int64)] # 最终获得每个训练点在哪个分区中

# 最后在训练时对nodeInfo[trainids] = trainIdsInPart

In [17]:
torch.max(trainIdsInPart)

tensor(7)

In [18]:
trainIdsInPart.shape

torch.Size([1207179])

In [22]:
trainIdsInPart.cpu().numpy().tofile("partinfo.bin")

In [19]:
ans = torch.bincount(trainIdsInPart)

In [23]:
trainIdsInPart

tensor([0, 6, 4,  ..., 3, 0, 2])